In [3]:
%matplotlib
import numpy as np
import csv
import json

Using matplotlib backend: <object object at 0x0000017507EBF510>


In [4]:
def inScattering(sctData):
    
    
    
    for i in range(len(sctData)):
        for j in range(len(sctData[0])):
            if i == j:
                sctData[i][j] = 0
    
    return sctData

In [5]:
def outScattering(sctData):
    
    outScat = np.zeros_like(sctData)
    
    for j in range(len(sctData[0])):
        sum = 0
        for i in range(len(sctData)):
            if i != j:
                sum += sctData[i][j]
                
        
        outScat[j][j] = sum 
        
    return outScat
                

In [6]:
def readCSV(filename):
    
    absorptions = []
    fissions = []
    chis = []
    SctData = []
    
    with open(filename, 'r') as file:
        
        csvreader = csv.reader(file)
        next(csvreader)#Title for Data
        next(csvreader)#Titles of Cross-Sections
        i=0
        
        
        for row in csvreader:
            #print(row)
            try:
                absorptions.append(float(row[1]))
                fissions.append(float(row[2]))
                chis.append(float(row[3]))
                i+=1
            except:
                
                break
        
        next(csvreader) #skips straight to next data
        
            
        length = len(absorptions)+1
        #print(length)
        
        for row in csvreader:
            #print(row)
            
            SctData.append([float(number) for number in row[1:length]])
            
    #print(absorptions)
    #print(fissions)
    #print(chis)
    #print(SctData)
    
    absMatr = np.diag(absorptions)
    
   
    
    fissMatr = np.zeros((len(fissions),len(fissions)))
    for i in range(0, len(fissMatr)):
        for j in range(0,len(fissMatr[0])):
            fissMatr[i][j] = fissions[j]*chis[i]
    
    
    
    #print(fissMatr)
    
    
    
    
    data = {}
    data["absorption"] = absMatr.tolist()
    data["fission"] = fissMatr.tolist()
    
    data["inScat"] = inScattering(SctData)
    data["outScat"] = outScattering(SctData).tolist()
    
    
    
    """
    data["inScat"] = np.array()
    data["outScat"] = np.array(SctData).tolist()
    """
    
    return data

In [7]:
def getMigration(absorps, outScat, inScat):
    
    migration = absorps + outScat - inScat
    
    return migration

In [8]:
def powerIterate(B_init, value):
    
    
    k_prev = 1
    s = (len(B_init), 1)
    
    phi_prev = np.ones(s)
    
    
    
    for i in range(0, value):
        
        phi_prev_num = np.matmul(B_init, phi_prev) #numerator of eq calculating new phi
        
        phi_new = phi_prev_num / np.linalg.norm(phi_prev_num) #values for next iteration
         
        phi_new_num = np.matmul(B_init, phi_new)# B times new flux
        
        k_new_num = np.matmul(np.transpose(phi_new_num), phi_new)#numerator for new  k value
        k_new_den = np.matmul(np.transpose(phi_new), phi_new)#denominator for new k value
        
        k_new = k_new_num / k_new_den
        
        k_prev = k_new
        
        phi_prev = phi_new
        

    return k_prev, phi_prev    

In [9]:
def main(filename):
    
    data = readCSV(filename)
    
    outScat, inScat = data["outScat"], data["inScat"]
    
    absorps = data["absorption"]
    fissions = data["fission"]
    
    migration = getMigration(np.array(absorps), np.array(outScat), np.array(inScat))
    
    B_init = np.matmul(np.linalg.inv(migration), np.array(fissions))
    #Calculating all eigenvalues using computer package
    eigenvals, eigenvectors = np.linalg.eig(B_init)
    
    #print(eigenvals, eigenvectors)  

    #get rid of complex in both
    eigenvals = eigenvals.tolist()
    eigenvectors = eigenvectors.tolist()
    

    eigenvals = [i.real for i in eigenvals]
    for i in range(len(eigenvectors)): 
        for j in range(len(eigenvectors[0])):
        
            eigenvectors[i][j] = eigenvectors[i][j].real 
    
    
    #calculate largest eigenval/eigenvector 
    k_final, phi_final = powerIterate(B_init, 2)#2 iterations


    eigs = {}
    eigs["valsComp"] = eigenvals #from the linalg package
    eigs["vecsComp"] = eigenvectors
    eigs["valPower"] = k_final.tolist()
    eigs["vecPower"] = phi_final.tolist()
    
    return eigs

In [10]:
def writeToOutput(inputFile):
    eigs = main(inputFile)
    inputData = readCSV(inputFile)
    
    
    
    outputFile = inputFile[5:-4] + ".txt"
    
    with open(outputFile, 'w+') as f:
        json.dump([inputData, eigs], f)

In [11]:
main("InputDataCSVGroup8.csv")
#writeToOutput("InputDataCSVGroup2.csv")

{'valsComp': [0.0,
  1.090030997354204,
  -1.2876327247127883e-17,
  3.764836114814617e-17,
  7.69729459116133e-18,
  -3.0242820992333182e-18,
  -3.0242820992333182e-18,
  -1.1623459657392265e-18],
 'vecsComp': [[-0.9827338208670577,
   -0.2615872477631111,
   -0.8731230399709816,
   -0.47358393515808317,
   -0.17662874226509687,
   0.8062464389519399,
   0.8062464389519399,
   -0.9259644552977211],
  [0.06596854844762812,
   -0.3441239021957947,
   0.31374808701281764,
   0.7578197001004813,
   0.09130053005236298,
   -0.08978160258237,
   -0.08978160258237,
   0.10394409262965572],
  [0.14990533908725992,
   -0.7819788590262128,
   -0.15431899568033036,
   -0.17288337936083917,
   -0.96916214011159,
   0.3898326091167121,
   0.3898326091167121,
   -0.3507941140162691],
  [0.053744049615705454,
   -0.2803549950510686,
   0.337303585260536,
   -0.15307281971227932,
   -0.10850224004426556,
   -0.1156674668084754,
   -0.1156674668084754,
   0.06129817227759667],
  [0.0427996373029088,
 